In [ ]:
! pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [ ]:
def get_editions (path='https://eurovision.tv/events'):
    contents = requests.get(path, headers = headers).content.decode()
    soup = BeautifulSoup(contents, 'html.parser')
    
    editions = []
    for row in soup.findAll('a'):
        if "https://eurovision.tv/event/" in str(row):
            editions.append(row['href'])
    
    return editions

In [ ]:
editions = get_editions()

In [ ]:
def get_countries (path):
    contents = requests.get(path, headers = headers).content.decode()
    soup = BeautifulSoup(contents, 'html.parser')
    
    countries = []
    for row in soup.findAll('table')[0].tbody.findAll('tr'):
        first_column = row.findAll('td')[1].findAll('span')[1].contents[0]
        countries.append("-".join(first_column.lower().split()))
    
    return countries

In [ ]:
def points_given(edition, country, countries, type_competition):
    contents = requests.get(edition + '/'+ type_competition +'/results/' + country, headers = headers).content.decode()
    soup = BeautifulSoup(contents, 'html.parser')
    points = []
    for table in soup.findAll('table'):
        type_of_points = table.thead.findAll('th')[0].contents[0].replace('\n','')
        if 'given' in type_of_points:
            for row in table.tbody.findAll('tr'):
                if len(row.findAll('td'))>1:
                    point = int(row.findAll('td')[0].contents[0].replace('\n','')) 
                    country_to = str(row.findAll('td')[1].contents[-1]).replace('\n','').lower()
                else:
                    country_to = str(row.findAll('td')[0].contents[-1]).replace('\n','').lower()
                points.append([edition.split('/')[-1], int(edition.split('/')[-1].split('-')[-1]), type_of_points, country, country_to, point])
    return points

In [ ]:
result = []
for edition in editions:
    print(edition)
    if int(edition.split('-')[-1])>2003:
        time.sleep(5)
        countries = get_countries(edition+"/grand-final")

        for country in countries:
            print(country)
            time.sleep(1)
            result.extend(points_given(edition, country, countries,'grand-final'))
    elif int(edition.split('-')[-1])>1956:
        countries = get_countries(edition+"/final")

        for country in countries:
            print(country)
            time.sleep(1)
            result.extend(points_given(edition, country, countries, 'final'))

In [ ]:
result = pd.DataFrame(result, columns=['Edition','Year','Points type','From','To','Points'])

In [ ]:
result.to_csv('Eurovision_1957-2021.csv')